In [0]:
############################################3 PArt 2 - Language Modelling ###############################3

#Importing needed modules
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandasql as ps

In [0]:
#Dataset Fetch and creating huge Positive, negative and Mixed strings
X_Y_full = pd.read_csv('/content/drive/My Drive/IMDB Dataset.csv')
len(X_Y_full)
positive =''
negative =''
mixed = ''
#23156374
def final_pop(x):
    global positive ;
    global negative ;
    global mixed;
    if(len(positive)==3156374):    ###### experimented with larger values and smaller values based on the resources of computation available
      return 1
    else:
      if(x['sentiment'] == 'positive'):
        positive=positive+x['review']
      else:
        negative=negative+x['review']
    mixed = mixed+x['review']
X_Y_full.apply (lambda row: final_pop(row), axis=1)



In [0]:
#sampling from the entire set
print(len(positive))
extract = 0.7 # tested different values
#text = text.lower()
text = positive[:int(extract*len(positive))] 
## change to negative/mixed for other models
## Kept as positive as it is the best model

In [5]:
#Dataset preparation with index to characters, characters to index
dict_1 = sorted(set(text.lower()))
print(dict_1)
print(len(text))

# Create mappings between vocab and numeric indices
char2idx_dict = {u:i for i, u in enumerate(dict_1)}
idx2char_dict = np.array(dict_1)

# Map all the training text over to a numeric representation
text_as_int_dict = np.array([char2idx_dict[c] for c in text.lower()])
#print(dict_1)
#print(idx2char_dict)
print(len(text_as_int_dict))

['\x08', '\t', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x80', '\x84', '\x85', '\x91', '\x95', '\x96', '\x97', '\x9a', '\xa0', '¡', '£', '¨', '«', '®', '´', '·', '»', '½', '¿', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ó', 'ô', 'õ', 'ö', 'ø', 'ú', 'ü', 'ý', 'ğ', 'ı', 'ō', '–', '‘', '’', '“', '”', '…']
14423292
14423292


In [0]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
print(examples_per_epoch)

159804


In [0]:
# Create Dataset with sequences of proper sequence length
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int_dict)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [0]:
#Dataset input as word at t, target as word at t+1
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)
print(dataset)

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>


In [0]:
#Split Dataset into batches
BATCH_SIZE = 134
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [0]:
#Model Design
embedding_dim = 50
rnn_units = 2048
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(dict_1), embedding_dim,
                          batch_input_shape=[BATCH_SIZE, None]))

model.add(layers.SimpleRNN(rnn_units,return_sequences=True))
model.add(tf.keras.layers.Dense(len(dict_1)))
model.summary()

In [0]:
#Define Loss Function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
model.compile(optimizer='adam', loss=loss)

In [0]:
#Checkpointing - saving model weights
import os
checkpoint_dir = '/content/drive/My Drive/check_rnn_lm'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True, 
    save_best_only=True)

directory not used yet.


In [0]:
#Model Training
history = model.fit(dataset, epochs=5,callbacks=[checkpoint_callback],verbose=1)
print("training complete.")

Epoch 1/2
1192/1192 [==============================] - 144s 121ms/step - loss: 2.3596
Epoch 2/2
1192/1192 [==============================] - 144s 121ms/step - loss: 1.8058
training complete.


In [0]:
#LOading the saved weights for testing the model with novel sampling so we give batch size as 1 here
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(len(dict_1), embedding_dim,
                          batch_input_shape=[1, None]))
model.add(layers.SimpleRNN(rnn_units,return_sequences=True))
model.add(tf.keras.layers.Dense(len(dict_1)))
model.add(layers.SimpleRNN(rnn_units,return_sequences=True))
model.add(tf.keras.layers.Dense(len(dict_1)))
model.load_weights(tf.train.latest_checkpoint('/content/drive/My Drive/check_rnn_lm'))
model.build(tf.TensorShape([1, None]))

In [0]:
#generate text function which samples out sentences
def generate_text(model, start_string):

    # Number of characters to generate
    num_generate = 200

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx_dict[s.lower()] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    text_ids_gen = []
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        #print(predictions)
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        #print(predicted_id)

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_ids_gen.append(predicted_id)
        text_generated.append(idx2char_dict[predicted_id])

    return (start_string + ''.join(text_generated))

In [0]:
print(generate_text(model, start_string="The "))

The powat cones ort bus omothaunseas mendof d b arnce e orofo a. s pa />br d. we thecon hesenda r atspllie ting cls a te totorngr--lle on, by ang>t cruley. thodan fusecama re anghas hearemavekispeverowa h


In [0]:
#Applying LSTM for the model
m = tf.keras.Sequential()
m.add(tf.keras.layers.Embedding(len(dict_1), embedding_dim,
                          batch_input_shape=[BATCH_SIZE, None]))
m.add(tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))
m.add(tf.keras.layers.Dense(len(dict_1)))

m.summary()

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

m.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/My Drive/check_lstm_LM'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True,
    save_best_only=True)





Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (134, None, 50)           6350      
_________________________________________________________________
lstm (LSTM)                  (134, None, 2048)         17195008  
_________________________________________________________________
dense_2 (Dense)              (134, None, 127)          260223    
Total params: 17,461,581
Trainable params: 17,461,581
Non-trainable params: 0
_________________________________________________________________


In [0]:
#Training LSTM model and saving it
history = m.fit(dataset, epochs=5,callbacks=[cb],verbose=1)
print("training complete.")
m.save('/content/drive/My Drive/Language_Modeling_positiveReviews.h5')

Epoch 1/5
1192/1192 [==============================] - 353s 296ms/step - loss: 1.8215
Epoch 2/5
1192/1192 [==============================] - 353s 296ms/step - loss: 1.3033
Epoch 3/5
1192/1192 [==============================] - 353s 296ms/step - loss: 1.2230
Epoch 4/5
1192/1192 [==============================] - 353s 296ms/step - loss: 1.1775
Epoch 5/5
1192/1192 [==============================] - 353s 296ms/step - loss: 1.1441
training complete.


In [0]:
#Loading the saved weights to test the model with batch size as 1
model_lstm = tf.keras.Sequential()
model_lstm.add(tf.keras.layers.Embedding(len(dict_1), 50,
                          batch_input_shape=[1, None]))
model_lstm.add(tf.keras.layers.LSTM(2048,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))
model_lstm.add(tf.keras.layers.Dense(len(dict_1)))

model_lstm.load_weights(tf.train.latest_checkpoint('/content/drive/My Drive/check_lstm_LM+ve'))
model_lstm.build(tf.TensorShape([1, None]))


In [10]:
#generating text from the model loaded with saved weights
print(generate_text(model_lstm, start_string=u"bad "))

bad how the boy wes david dead as allowing day -- freedom. maxline was delivered at times for in a white catwoman (givin shaw-mal) maker a surprise match. streep in being in over all: in any friend as he 


In [12]:
#Another method of loading the trained model instead of loading the weights
import tensorflow as tf 
classifierLoad = tf.keras.models.load_model('/content/drive/My Drive/Language_Modeling_positiveReviews.h5')

In [13]:
#Generating text from the loaded model
print(generate_text(classifierLoad, start_string=u"bad "))

bad perfect esecationalism and desolated dark-comedy from early 90's and that fall in love amount of adults as well. on the only two departments of the film could be a big fan of the happy ending only it 


In [0]:
################## Statistical model  ##############
import nltk
from nltk.util import ngrams
from nltk.lm import MLE
from nltk import word_tokenize
# we need to download a special component that is used by the tokenizer below -- don't worry about it. 
nltk.download('punkt')
!pip install -U nltk==3.4
!pip install nltk


In [0]:
#Preparing the Training testing set with 8:2 split
review_col = X_Y_full['review'].str.lower()
review = np.asarray(review_col)
!pip install pandasql
import pandasql as ps
review_alone = ps.sqldf('select review from X_Y_full')
print(len(review_alone))
train,test = np.split(review_alone.sample(frac=1), [int(.8*len(X_Y_full))])
#Maintain a shorter length on test data for testing perplexity
test = ps.sqldf('select substr(review,1,100) as review from  test limit 100')
review = train['review'].to_list()

#Tokenise the sentences
texts = []
for s in review:
    print(s)
    texts.append(word_tokenize(s))
print(list(ngrams(texts[0], n=2)))

#Preparing ngrams
from nltk.lm.preprocessing import padded_everygram_pipeline
train, vocab = padded_everygram_pipeline(3, texts) #### played with 3 and 2 values

#model for trigrams
model = MLE(3) # changed to 2 and experimented
model.fit(train, vocab)

#statistical test
#sentence generation examples, played eith different lengths
word_list = model.generate(15, random_seed=50)
#score and counts
print(model.counts['bad'])
print(model.score('bad','movie is'.split()))
print(model.counts[['bad']]['movie'])

#computes perplexity of the unigram model on a testset  
def perplexity(testset, model,ngrams):
        testset = testset['review']
        tokenized_text = list(map(str.lower, nltk.tokenize.word_tokenize(testset))) 
        test_data = nltk.ngrams(tokenized_text, n=ngrams,  pad_right=False, pad_left=False, left_pad_symbol="<s>", right_pad_symbol="</s>")
        print(test_data)
        perplexity = 1
        #global N;
        
       # for ngram in test_data])
        #print(test_data[-1])
        temp=1
        N=0
        prob=1
        for ngram in test_data:
            print ("MLE Estimates:", ((ngram[-1], ngram[:-1]),model.score(ngram[-1], ngram[:-1])))
            #global N;
            N += 1
            str1=' '
            follow = str1.join(ngram[:-1])
            #print(follow)
            #print(model.score(ngram[-1], follow.split()))
            if(model.score(ngram[-1], follow.split())>0 and model.score(ngram[-1], follow.split())!=0.0):
                prob = prob * (model.score(ngram[-1], follow.split()))
                print(prob)   
            else:
                prob = prob * 0.00001
            temp=N 
        multi = 1/prob    
        perplexity = pow(multi, 1/temp) 
        return perplexity

In [0]:
#perplexity test
testset1 = "does this film suck" #sample
ngrams=3 # tested for 2
# adding a perplexity column with values to test set to review later
test['perplexity'] = test.apply (lambda row: perplexity(row, model,ngrams),axis=1)
print(test)

In [0]:
######################## Word Embedding ##########################
extract = 0.2
#text = text.lower()
text = positive[:int(extract*len(positive))]
vocab_words = []
words = text.split(" ")
for j in words:
        vocab_words.append(j.lower())
        
vocab_unique_words = sorted(set(vocab_words))
print(len(vocab_unique_words))  



In [0]:
# Create mappings between vocab and numeric indices
char2idx_word = {u:i for i, u in enumerate(vocab_unique_words)}
idx2char_word = np.array(vocab_unique_words)

# Map all the training text over to a numeric representation
text_as_int_word = np.array([char2idx_word[c] for c in text.lower().split(' ')])
#print(dict_1)
#print(idx2char_dict)
#print(len(text_as_int_dict))

In [0]:
# The maximum length sentence we want for a single input in characters
seq_length_word = 20
examples_per_epoch_word = len(text.split(" "))//(seq_length_word+1)
print(examples_per_epoch_word)

38584


In [0]:
# Create training examples / targets
char_dataset_word = tf.data.Dataset.from_tensor_slices(text_as_int_word)
sequences_word = char_dataset_word.batch(seq_length_word+1, drop_remainder=True)


In [0]:
#Dataset Preparation
def split_input_target_word(chunk):
    print(chunk[:-1])
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset_word = sequences_word.map(split_input_target_word)
print(dataset_word)

Tensor("strided_slice:0", shape=(20,), dtype=int64)
<MapDataset shapes: ((20,), (20,)), types: (tf.int64, tf.int64)>


In [0]:
#Model Design
BATCH_SIZE = 1000
BUFFER_SIZE = 10000
dataset_word = dataset_word.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
embedding_dim = 256
rnn_units = 2048
m2 = tf.keras.Sequential()
m2.add(tf.keras.layers.Embedding(len(vocab_unique_words), embedding_dim,
                          batch_input_shape=[BATCH_SIZE, None]))
m2.add(tf.keras.layers.LSTM(rnn_units,
                    return_sequences=True,
                    stateful=True,
                    recurrent_initializer='glorot_uniform'))
m2.add(tf.keras.layers.Dense(len(vocab_unique_words)))

m2.summary()



In [0]:
#Loss function
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

m2.compile(optimizer='adam', loss=loss)

# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/My Drive/check_lstm_LM_words'
import shutil
try:
    shutil.rmtree(checkpoint_dir)
except:
    print("directory not used yet.")

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_")

cb=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    monitor='loss',
    save_weights_only=True,
    save_best_only=True)




directory not used yet.


In [0]:
#Training word embeddings model
history = m2.fit(dataset_word, epochs=2,callbacks=[cb],verbose=1)
print("training complete.")